# Unshuffle a sequence of frames 

This notebook is intended to xx. 

1. Preperation of the data for analysis
2. Analysis code 
3. Testing and validation


In [35]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageSequence

%matplotlib inline

## Load and display sequences 
This is meant to prepare the data for the analysis. This includes loading in the movie, shuffling and then displaying them. 



In [59]:
gif = load_gif_and_shuffle(filename)

In [58]:
def shuffle(sequence, seed_value=42):
    """Randomly shuffle a sequence of numbers."""
    np.random.seed(seed_value)
    return list(np.random.permutation(sequence))

def load_gif(filename):
    """Load a sequence of images (for now in GIF format) and return an array."""
    output = {}
    gif = Image.open(filename)
    for i, frame in enumerate(ImageSequence.Iterator(gif)):
        output[i] = np.array(frame.convert("RGB"))
        #print("Saved image: {}".format(i)) 
    return output

def load_gif_and_shuffle(filename, frame0=True):
    
    output = {}
    gif = load_gif(filename)
    k = list(gif.keys())
    
    if frame0:
        new_k = [0] + shuffle(k[1:])
    else:
        new_k = shuffle(k)
    for i,k in enumerate(new_k):
        output[i]=gif[k]
    return output

def display_sequence(sequence): 
    """Display sequence"""

# Load the data:
filename = "/Users/melisatallis/Documents/Github/falco_problemita/data/race_horse.gif"
# Make a copy of the gif in case it is overwritten

In [ ]:
sequence = load_gif(filename)

**Figure:** An example that has been shuffled for this analysis. 

## Next, compare images of sequence
An interesting question is how do we compare frames without caring about the content inside of each frame. Is there a way to rank the similarity between two images.  

## Calculating similarity

To speed up the analysis, we start by comparing images for a single color channel. 

Afterwards, we repeat the analysis on the remaining channels. We´ll end up with three new sequences, ordered using information from their corresponding channel. 


In [ ]:
new_sequence = {} # keys: r,b,g
for i in sequence:
    print()

In [ ]:
def ssm(arr):
    return 



In [ ]:
for 